In [12]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
from utils import *
from langchain.schema import Document

load_dotenv()

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

all_texts = load_pdfs_from_folder("data")
all_text_chunks = chunk_texts(all_texts)

documents = [
    Document(page_content=chunk, metadata={"source": "manual_input"}) 
    for chunk in all_text_chunks
]


db = FAISS.from_documents(documents, embeddings)
db.save_local("faiss_index") 


In [17]:
from langchain.chat_models import ChatOpenAI

load_dotenv()

# embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model_name="gpt-4o")

db = FAISS.load_local(
    "faiss_index", 
    embeddings, 
    allow_dangerous_deserialization = True
    )

def ask(query):
    docs = db.similarity_search(query, k=3)
    context = "\n\n".join(doc.page_content for doc in docs)
    
    prompt = f"""Answer the question based only on the context below.

Context:
{context}

Question: {query}"""

    response = llm.predict(prompt)
    return response

if __name__ == "__main__":
    q = "what is insurance and how does it work?"
    print("\nAnswer:", ask(q))
    q = "What are the main types of insurance?"
    print("\nAnswer:", ask(q))



Answer: Insurance is a contract, known as a policy, in which a policyholder receives financial protection or reimbursement against losses from an insurance company. The company pools the risks of its clients to make payments more affordable for the insured. This financial protection can cover losses resulting from accidents, injuries, property damage, or liability for damage or injury caused to a third party. Essentially, insurance helps to protect individuals and their families against unexpected financial costs, potential debts, and the risk of losing assets.

Answer: The main types of insurance are accident and health insurers, property and casualty insurers, and financial guarantors. The most common types of personal insurance policies are auto, health, homeowners, and life insurance.
